In [1]:
!pip install transformers torch pandas scikit-learn

In [2]:
import pandas as pd

In [5]:
df = pd.read_csv('anime.csv')
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [8]:
df.shape

(12294, 7)

In [15]:
df = df.dropna()
df['description'] = df['name'] +' '+ df['genre'] + ' ' +df['type']+' episodes: '+ df['episodes'].astype(str) + ' rating: '+ df['rating'].astype(str)
df.head()

,anime_id,name,genre,type,episodes,rating,members,description
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,"Kimi no Na wa. Drama, Romance, School, Superna..."
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,"Fullmetal Alchemist: Brotherhood Action, Adven..."
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262,"Gintama° Action, Comedy, Historical, Parody, S..."
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572,"Steins;Gate Sci-Fi, Thriller TV episodes: 24 r..."
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,"Gintama&#039; Action, Comedy, Historical, Paro..."


In [16]:
df.shape

(12017, 8)

In [17]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

In [18]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

def get_embeddings(sentences):
  encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

  with torch.no_grad():
      model_output = model(**encoded_input)

  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

  sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

  return sentence_embeddings

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [19]:
sentences = ['Some great movie', 'Another funny movie']
result = get_embeddings(sentences)
print("Sentence embeddings:")
print(result)

Sentence embeddings:
tensor([[-1.0470e-01,  1.3669e-02, -3.7724e-02,  3.6088e-02, -3.0928e-02,
          3.3181e-02,  6.8035e-02, -4.1169e-02,  1.1926e-01, -4.1912e-02,
          3.9141e-02,  1.5677e-02, -3.6529e-02,  2.3321e-02, -3.2273e-02,
         -5.2346e-03,  3.8707e-02,  6.9843e-03, -2.7907e-02, -8.3663e-02,
         -5.1465e-02,  3.7350e-02,  3.9144e-02,  4.7031e-02, -1.6407e-01,
          2.1509e-02,  2.6223e-02,  1.4407e-03, -9.1594e-02, -3.3018e-02,
         -3.2289e-02,  6.4941e-02, -9.3422e-02,  3.0320e-02, -3.7254e-02,
          1.1184e-02,  3.4294e-02,  2.5122e-03, -2.7599e-02, -1.9730e-02,
         -3.3302e-02, -8.9150e-02,  9.6148e-02, -1.3116e-02,  3.5138e-02,
         -3.0958e-02, -2.3608e-02, -1.4274e-02,  4.8779e-02,  3.8363e-02,
         -2.9066e-02,  8.3276e-02, -5.3506e-02, -7.7774e-03,  4.9438e-02,
         -4.2659e-02, -5.9170e-02, -9.7170e-02,  3.5791e-02, -1.4385e-01,
          6.4405e-02, -3.7883e-02,  2.8620e-02, -5.5651e-02,  7.5088e-02,
          2.2148e

#Using Hugging Face Package for embedding sentence transformer

In [20]:
!pip install -U sentence-transformers

In [21]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

anime_embeddings = model.encode(df['description'].tolist())

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

def get_recommendations(query, embeddings, df, top_n=5):
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, embeddings)
    top_indices = similarities[0].argsort()[-top_n:][::-1]
    return df.iloc[top_indices]

In [24]:
# query = input("What are you searching to watch: ") #"Funny anime I can watch with friends"
# recommendations = get_recommendations(query, anime_embeddings, df)
# print(recommendations[['name', 'genre']])

In [31]:
# from IPython.display import display
# import ipywidgets as widgets

# query = input("What are you searching to watch: ")  # e.g., "Funny anime I can watch with friends"
# recommendations = get_recommendations(query, anime_embeddings, df)

# # Create interactive dropdown
# dropdown = widgets.Dropdown(
#     options=[(row['name'], idx) for idx, row in recommendations.iterrows()],
#     description='Select:',
#     style={'description_width': 'initial'},
#     layout=widgets.Layout(width='50%')
# )

# output = widgets.Output()

# def on_select(change):
#     output.clear_output()
#     with output:
#         idx = change['new']
#         row = recommendations.loc[idx]
#         print(f"**Name:** {row['name']}")
#         print(f"**Genre:** {row['genre']}")

# dropdown.observe(on_select, names='value')

# display(dropdown, output)


In [35]:
from google.colab import data_table
from IPython.display import display

# Enable interactive tables (scrollable, sortable)
data_table.enable_dataframe_formatter()

query = input("What are you searching to watch: ")  # e.g., "Funny anime I can watch with friends"
recommendations = get_recommendations(query, anime_embeddings, df)

# Select only required columns
result = recommendations[['name', 'genre']].reset_index(drop=True)

# Display as interactive table
display(result)


What are you searching to watch: Funny anime I can watch with friends


,name,genre
0,Heisei Harenchi Gakuen,"Comedy, Hentai, School"
1,Oshiete! Galko-chan,"Comedy, School, Slice of Life"
2,Sentou Yousei Shoujo Tasukete! Mave-chan,"Comedy, Parody, Sci-Fi, Shounen, Super Power"
3,Rikujou Bouei-tai Mao-chan,"Comedy, Kids, Magic, Military, Sci-Fi"
4,Anime TV de Hakken! Tamagotchi,"Comedy, Fantasy, Kids, Slice of Life"
